# OCR con DOCTR

DocTR (Document Text Recognition) es una librería open-source de OCR basada en deep learning creada por Mindee.
Proporciona un pipeline end-to-end con dos componentes principales:

- Detector de texto: Delimita automáticamente las regiones que contienen palabras o líneas, sin necesidad de utilizar otras herramientas (como hicimos con PyMuPDF para correr Tesseract). El dectector analiza la imagen, localiza dónde hay palabras/líneas y dibuja “cajas” (bounding boxes).

- Reconocedor de texto: Transforma cada región en caracteres, para ello recorta cada caja y convierte los píxeles en caracteres (CNN + RNN/Transformer).

Seleccionamos este modelo porque tiene alto recall en documentos reales, maneja  márgenes, sellos, firmas y layouts irregulares que aparecen en cartas documento escaneadas. Documentación disponible en https://mindee.github.io/doctr/index.html

Su instalación es bastante sencilla, solo hay que importar las librerías en la terminal. Para mayor información, ver archivo "Readme_DOCTR"

1) Importamos librerías

In [ ]:
import os
import torch
from doctr.io import DocumentFile
from doctr.models import ocr_predictor


d:\entornos\trocr_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Definimos las rutas necesarias

In [ ]:
IN_DIR  = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\CDs_Ejemplo"
OUT_DIR = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\Extraccion_DOCTR"
os.makedirs(OUT_DIR, exist_ok=True)

3. Definimos los parámetros del modelo

In [3]:
DET_ARCH          = "db_mobilenet_v3_large"   # detector rápido y preciso :contentReference[oaicite:0]{index=0}
RECO_ARCH         = "sar_resnet31"            # reconocedor robusto a tildes :contentReference[oaicite:1]{index=1}
DET_BS, RECO_BS   = 4, 512                    # batch sizes seguros para 6 GB VRAM :contentReference[oaicite:2]{index=2}
ASSUME_STRAIGHT   = False                     # deja que detecte rotación :contentReference[oaicite:3]{index=3}
DEVICE            = "cuda" if torch.cuda.is_available() else "cpu"
FP16              = DEVICE == "cuda"          # medio flotante sólo en GPU

d:\entornos\trocr_env\lib\site-packages\torch\cuda\__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


4. Instanciamos el modelo y configuramos los parámetros

In [4]:
model = ocr_predictor(
    det_arch=DET_ARCH,
    reco_arch=RECO_ARCH,
    pretrained=True,
    det_bs=DET_BS,
    reco_bs=RECO_BS,
    assume_straight_pages=ASSUME_STRAIGHT,
    detect_orientation=True,          # clasificador de orientación de página
    straighten_pages=True,            # deskew suave
    preserve_aspect_ratio=True,       # evita distorsión
    symmetric_pad=True,               # paddings parejos
    resolve_blocks=False,             # sólo texto plano
).to(DEVICE)

if FP16:
    model = model.half()             #  reduce VRAM :contentReference[oaicite:4]{index=4}

d:\entornos\trocr_env\lib\site-packages\doctr\models\utils\pytorch.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(archive_path, map_location="c

5. Definimos una función para convertir en texto plano

In [5]:
def plano(p):
    return "\n".join(
        " ".join(w["value"] for w in l.get("words", []))
        for b in p.get("blocks", [])
        for l in b.get("lines", [])
    )

6. Recorremos todos los archivos del repositorio y aplicamos el OCR

In [6]:
for fname in os.listdir(IN_DIR):
    if not fname.lower().endswith(".pdf"):
        continue
    pdf_path = os.path.join(IN_DIR, fname)
    try:
        # Rasteriza a ~300 dpi, escala = 2 (72 dpi × 2 ≈ 144 dpi por pulgada lógica de pdfium) :contentReference[oaicite:5]{index=5}
        doc = DocumentFile.from_pdf(pdf_path, scale=2)
        print(f"{fname}: {len(doc)} pág(s)")

        result = model(doc).export()["pages"]
        texto = []
        for i, page in enumerate(result, start=1):
            texto.append(f"\n--- Página {i} ---\n")
            texto.append(plano(page))

        txt_name = os.path.splitext(fname)[0] + ".txt"
        with open(os.path.join(OUT_DIR, txt_name), "w", encoding="utf-8") as f:
            f.write("".join(texto))

        print(f"✔ Guardado {txt_name}")
    except Exception as e:
        print(f"✖ Error en {fname}: {e}")

Belen_payway.pdf: 1 pág(s)
✔ Guardado Belen_payway.txt
Bruno23689270.pdf: 1 pág(s)
✔ Guardado Bruno23689270.txt
Bruno400212294002.pdf: 1 pág(s)
✔ Guardado Bruno400212294002.txt
Bruno400212294002_1.pdf: 1 pág(s)
✔ Guardado Bruno400212294002_1.txt
Coyle401547680601.pdf: 1 pág(s)
✔ Guardado Coyle401547680601.txt
EVERTEC30707869484.pdf: 1 pág(s)
✔ Guardado EVERTEC30707869484.txt
Galicia_Giuseppe.pdf: 1 pág(s)
✔ Guardado Galicia_Giuseppe.txt
hsbc_Aquino.pdf: 1 pág(s)
✔ Guardado hsbc_Aquino.txt
hsbc_Aquino2.pdf: 1 pág(s)
✔ Guardado hsbc_Aquino2.txt
hsbc_Aquino3.pdf: 1 pág(s)
✔ Guardado hsbc_Aquino3.txt
Mallo12980371.pdf: 1 pág(s)
✔ Guardado Mallo12980371.txt
Pantano28462989.pdf: 1 pág(s)
✔ Guardado Pantano28462989.txt
Santander_evertec.pdf: 1 pág(s)
✔ Guardado Santander_evertec.txt


## Estracción de variables con LLama 3

Vamos a seguir la misma lógica de extracción de variables que utilizamos con Tesseract

In [8]:
pip install requests tqdm pydantic

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 9.9 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)

   ---------------------------------------- 0/4 [typing-inspection]
   ---------- ----------------------------- 1/4 [pydantic-core]
   ---------- ----------------------------- 1/4 [pydantic-core]
   ------------------------------ --------- 3/4 [pydantic]
   ------------------------------ --------- 3/4 [pydantic]
   ------------------------------ --------- 3/4 [pydantic]
   ------------------------------ --------- 3/4 [pydantic]
   ------------------------------ -

In [9]:
from pydantic import BaseModel
from typing import Optional
from pydantic import ValidationError
import re, unicodedata
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from pathlib import Path
import json
import csv
import requests
from requests.exceptions import ReadTimeout

In [10]:
TXT_FOLDER = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\Extraccion_DOCTR"
OUTPUT_CSV = "entidades_extraidas_DOCTR.csv"
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 800
MAX_CHARS   = 9000

In [11]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str

In [12]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [13]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

In [14]:
if __name__ == "__main__":
    main()

TXT:  46%|████▌     | 6/13 [1:34:04<1:50:18, 945.53s/it] 

❌ EVERTEC30707869484.txt: Timeout de Ollama


TXT: 100%|██████████| 13/13 [1:59:02<00:00, 549.43s/it] 


✅ CSV generado en entidades_extraidas_DOCTR.csv (12 filas)
